In [ ]:
import numpy as np
import scipy as sc
from scipy import signal


In [ ]:
import wfdb
import matplotlib.pyplot as plt
from scipy.signal import spectrogram
from scipy.signal import periodogram
import numpy as np
from scipy import signal
import pandas as pd

# # 파일 경로 설정
# record_name = 'A00001'  # 파일의 경로를 제외한 기본 이름
directory_path = './training2017/'  # 필요한 경우 실제 파일 경로를 지정하세요.

# CSV 파일의 경로 지정
file_path = directory_path+'REFERENCE-original.csv'

# 파일 불러오기
data = pd.read_csv(file_path, header=None, names=['ID', 'Label'])
# 불러온 데이터의 처음 몇 줄을 출력


In [ ]:
data

In [ ]:
label_set = set(data['Label'])

In [ ]:
if 'A' in label_set:
    print(True)

In [ ]:
import os

path = "./spectrogram_img"
os.makedirs(path, exist_ok=True)

for label in label_set:
    sub_path = os.path.join(path, label)
    os.makedirs(sub_path, exist_ok=True)

In [ ]:
def data_read(record_name, directory_path = directory_path):
    # 데이터 로드
    record = wfdb.rdrecord(f"{directory_path}/{record_name}")
    signal = record.p_signal
    
    fs = record.fs
    return signal, fs

In [ ]:
import scipy.signal as sp_signal

def spectrogram(data, nperseg=64, noverlap=32, log_spectrogram = True):
    fs = 300
    f, t, Sxx = sp_signal.spectrogram(data, fs=fs, nperseg=nperseg, noverlap=noverlap)
    Sxx = np.transpose(Sxx,[0,2,1])
    if log_spectrogram == True:
        Sxx = abs(Sxx) # Make sure, all values are positive before taking log
        mask = Sxx > 0 # We dont want to take the log of zero
        Sxx[mask] = np.log(Sxx[mask])
    return f, t, Sxx

In [ ]:
import cv2
def cartesian_to_polar(img):
    height, width = img.shape[:2]

    max_radius = height
    output_size = 2 * max_radius

    center = output_size // 2

    # 2r x 2r 크기의 극 좌표 이미지 생성
    polar_img = np.zeros((output_size, output_size, 3), dtype=np.uint8) if len(img.shape) == 3 else np.zeros((output_size, output_size), dtype=np.uint8)

    for i in range(output_size):
        for j in range(output_size):
            dx = j - center
            dy = center - i

            r = np.sqrt(dx*dx + dy*dy)
            theta = (np.arctan2(dy, dx) + np.pi) % (2 * np.pi)  # 0 to 2pi

            theta_idx = int(theta / (2 * np.pi) * width)
            r_idx = int(r)

            if 0 <= r_idx < height:
                polar_img[i, j] = img[r_idx, theta_idx]

    # 보간 처리
    mask = polar_img[:, :] > 0
    polar_img = cv2.inpaint(polar_img, np.uint8(~mask), inpaintRadius=1, flags=cv2.INPAINT_TELEA)

    # 극 좌표 이미지를 256x256 크기로 리사이즈
    polar_img = cv2.resize(polar_img, (256, 256), interpolation=cv2.INTER_LINEAR)

    return polar_img

In [ ]:
import cv2
# import numpy as np
# def cartesian_to_polar(img, output_size=(256, 256)):
#     height, width = img.shape[:2]

#     max_radius = height
#     output_size = (output_size[0], output_size[1])  # 새로운 출력 크기

#     center = output_size[0] // 2

#     # 2r x 2r 크기의 극 좌표 이미지 생성
#     polar_img = np.zeros((output_size[0], output_size[1], 3), dtype=np.uint8) if len(img.shape) == 3 else np.zeros((output_size[0], output_size[1]), dtype=np.uint8)

#     for i in range(output_size[0]):
#         for j in range(output_size[1]):
#             dx = j - center
#             dy = center - i

#             r = np.sqrt(dx*dx + dy*dy)
#             theta = (np.arctan2(dy, dx) + np.pi) % (2 * np.pi)  # 0 to 2pi

#             theta_idx = int(theta / (2 * np.pi) * width)
#             r_idx = int(r)

#             if 0 <= r_idx < height:
#                 polar_img[i, j] = img[r_idx, theta_idx]

#     # 보간 처리
#     mask = polar_img[:, :] > 0
#     polar_img = cv2.inpaint(polar_img, np.uint8(~mask), inpaintRadius=1, flags=cv2.INPAINT_TELEA)

#     # 극 좌표 이미지를 256x256 크기로 리사이즈
#     polar_img = cv2.resize(polar_img, (256, 256), interpolation=cv2.INTER_LINEAR)

#     return polar_img


In [ ]:
def plot_fourier_transform(image):
    # 2D 푸리에 변환
    f_transform = np.fft.fft2(image)
    
    # 중심 이동
    f_transform_shifted = np.fft.fftshift(f_transform)
    
    # 로그 스케일로 변환
    magnitude_spectrum = np.log(np.abs(f_transform_shifted) + 1)
    
    # 결과 시각화
    plt.imshow(magnitude_spectrum, cmap='gray')
    plt.title('Magnitude Spectrum')
    plt.colorbar()
    plt.show()

In [ ]:
import scipy.signal as sp_signal

visual = 0
i = 0 
for fname, label in zip(data['ID'], data['Label']):

    # print(fname)
    if i == 0:
        signal, fs = data_read(fname)
        ts = signal[:,0]
        time = np.arange(0, len(ts))/fs

        # Now, let's create the combined plot
        
        if visual == 0:
            fig, axes = plt.subplots(4, 1, figsize=(15, 20))  # Adjust the figure size if needed
        # ECG time series
            axes[0].plot(time, ts, 'b')
            axes[0].set(xlabel='Time [s]', xlim=[0, time[-1]], xticks=np.arange(0, time[-1]+5, 10))
            axes[0].set(ylabel='Potential [mV]')
            axes[0].set_title('ECG Time Series', fontsize=15)


        f1, PSD = sp_signal.periodogram(ts, fs, 'flattop', scaling='density')
        if visual == 0:
            axes[1].plot(f1, PSD, 'b')
            axes[1].set(xlabel='Frequency [Hz]', ylabel='PSD')
            axes[1].set_title('Power Spectral Density (PSD)', fontsize=15)

        # Spectrogram without log transform

        # Convert ECG into spectrograms without and with log transform
        Sx = spectrogram(np.expand_dims(ts, axis = 0), log_spectrogram = False)[2]
        Sx_log = spectrogram(np.expand_dims(ts, axis = 0), log_spectrogram = True)[2]

        # Get the frequency and time axes
        f, t, _ = spectrogram(np.expand_dims(ts, axis = 0), log_spectrogram = False) 
        if visual == 0:
            axes[2].imshow(np.transpose(Sx[0]), aspect='auto', cmap='jet', origin='lower')
            axes[2].set_title('Spectrogram without log transform', fontsize=15)
            axes[2].set(xlabel='Time [s]', ylabel='Frequency [Hz]')
        
            axes[3].imshow(np.transpose(Sx_log[0]), aspect='auto', cmap='jet', origin='lower')
            axes[3].set_title('Spectrogram with log transform', fontsize=15)
            axes[3].set(xlabel='Time [s]', ylabel='Frequency [Hz]')

            # Main title for the entire figure
            fig.suptitle(label + '_' + fname, fontsize=20, y=1.02)
            fig.savefig( path + '/'+ label + '/' + fname + '_monitor.png')
            plt.tight_layout()
            plt.show()

        spectrogram_image_data = np.transpose(Sx[0])
        log_spectrogram_image_data = np.transpose(Sx_log[0])
        # plt.imsave( label + '_' + fname + '_spectrogram.png', spectrogram_image_data, cmap='jet')
        plt.imsave( path + '/'+ label + '/' + fname + '_log_spectrogram.png', log_spectrogram_image_data, cmap='jet')

        polar_img = cartesian_to_polar(np.flipud(log_spectrogram_image_data))
        # polar_img_2d = np.squeeze(polar_img)  # 차원이 1인 경우 제거

        print(log_spectrogram_image_data.shape)
        plt.imsave(path + '/' + label + '/' + fname + '_polar.png', polar_img, cmap='jet')

        plot_fourier_transform(polar_img)

    i+=0

In [ ]:
log_spectrogram_image_data

In [ ]:
np.any(log_spectrogram_image_data > 0)

In [ ]:
# 최소 값을 찾아서 모든 값에 더함
offset_value = np.abs(np.min(log_spectrogram_image_data))
positive_data = log_spectrogram_image_data + offset_value

# 시각화
plt.imshow(log_spectrogram_image_data, cmap='jet')
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(positive_data, cmap='jet')
plt.colorbar()
plt.show()

In [ ]:
# https://github.com/mikgroup/sigpy-mri-tutorial/blob/master/01-gridding-reconstruction.ipynb